In [1]:
%load_ext autoreload
%autoreload 2

%cd ..

/Users/rubenbroekx/Documents/Projects/radix-co2-reduction


# Sentinel-1

Script to experiment with [Sentinel-1](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S1_GRD?hl=en) data.

In [2]:
import folium
import ee

from src.radix_co2_reduction.earth_engine.session import start
from src.radix_co2_reduction.earth_engine.visualisation import create_map, show_polygon, show_point
from src.radix_co2_reduction.earth_engine.utils import create_bounding_box, to_polygon
from src.radix_co2_reduction.earth_engine.datasets import Sentinel1Collection

In [3]:
# Start a GEE session
start()

Enter verification code: 4/1AY0e-g5bhKnbb-MjEHZDO4xJ6wXdVMYTWorkneOpXlqhuxy60UWNLl0r_hw

Successfully saved authorization token.


In [17]:
# Specify the coordinates to investigate
LAT = 40.6281873302212
LNG = -89.4530439376831

In [19]:
# Choose region to investigate
box = to_polygon(create_bounding_box(
    lat=LAT,
    lng=LNG,
    offset=1_000,
))

## Manual

Manually create the library (directly from GEE).

In [20]:
# Create dataset
coll = ee.ImageCollection('COPERNICUS/S1_GRD').\
    filterDate('2020-01-01', '2020-12-31').\
    filterBounds(box).\
    select('VH').\
    filter(ee.Filter.eq('instrumentMode', 'IW')).\
    filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).\
    filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')).\
    filter(ee.Filter.eq('resolution', 'H'))

print(f"Dataset size:", coll.size().getInfo())

Dataset size: 30


In [21]:
# Create mosaic image
mosaic = coll.mosaic()

In [22]:
# Create image to show progress over seasons
spring = ee.Filter.date('2020-03-01', '2020-04-20');
lateSpring = ee.Filter.date('2020-04-21', '2020-06-10');
summer = ee.Filter.date('2020-06-11', '2020-08-31');

seasons = ee.Image.cat(
    coll.filter(spring).median(),
    coll.filter(lateSpring).median(),
    coll.filter(summer).median()
)

In [23]:
# Create a map visualisation
mp = create_map((LAT,LNG), zoom=14)

# Display dataset
mp.add_ee_layer(
    mosaic,
    {'min': -25., 'max': 1.},
    'Sentinel 1 - Mosaic',
)
mp.add_ee_layer(
    seasons,
    {'min': -25., 'max': 1.},
    'Sentinel 1 - Seasons',
)

# Display geometry information
mp = show_polygon(mp, box)
mp = show_point(mp, ee.Geometry.Point(LNG,LAT))

# Show map
mp.add_child(folium.LayerControl())
display(mp)

## Library

Use our own Earth Engine wrapper to load in the data.

In [24]:
sentinel1 = Sentinel1Collection()
sentinel1.load_collection(
    region=box,
    startdate='2020-01-01',
    enddate='2020-12-31',
)
print(f"Loaded images:", sentinel1.get_size())

Loaded images: 30


In [25]:
# Create a map visualisation
mp = create_map((LAT,LNG), zoom=12)

# Display dataset
mp = sentinel1.add_to_map(
    mp=mp,
    scheme='all',
)

# Display geometry information
mp = show_polygon(mp, box)
mp = show_point(mp, ee.Geometry.Point(LNG,LAT))

# Show map
mp.add_child(folium.LayerControl())
display(mp)

Adding images: 100%|██████████| 30/30 [02:24<00:00,  4.83s/it]


In [12]:
# Experiment with sampling
pixels = ee.FeatureCollection.randomPoints(
    region=box,
    points=1000,
    seed=42,
    maxError=0,
)
sample = sentinel1.sample(
    pixels=pixels,
)

Sampling Sentinel-1 SAR GRD: 100%|██████████| 30/30 [05:43<00:00, 11.46s/it]


In [13]:
values_vv, values_vh = [], []
for date,samples in sample.items():
    values_vv += [v for v in samples['SAR_VV'] if v]
    values_vh += [v for v in samples['SAR_VH'] if v]

In [14]:
min(values_vv), sum(values_vv)/len(values_vv), max(values_vv)

(0.2936660051345825, 0.7414523763856943, 1)

In [15]:
min(values_vh), sum(values_vh)/len(values_vh), max(values_vh)

(0.16655072569847107, 0.5972536142240468, 0.9392660856246948)

In [16]:
sample

{'2020-01-01': {'SAR_VV': [0.7406160235404968,
   0.7524768114089966,
   0.8027189373970032,
   0.772847592830658,
   0.7011041641235352,
   0.8071008920669556,
   0.8441389203071594,
   0.7787488102912903,
   0.9658522009849548,
   0.8099675178527832,
   0.7372224926948547,
   0.8166790008544922,
   0.7223438024520874,
   0.7055604457855225,
   0.8210759162902832,
   0.8093105554580688,
   0.7663224935531616,
   0.736892580986023,
   0.7819176316261292,
   0.7159522771835327,
   0.6930117607116699,
   0.8779926896095276,
   0.8609820008277893,
   0.7549324035644531,
   0.7990495562553406,
   0.7626915574073792,
   0.8171131610870361,
   0.8278170228004456,
   0.7833377718925476,
   0.7767715454101562,
   0.7716267704963684,
   0.7348324060440063,
   0.7515090703964233,
   0.8420872688293457,
   0.7764191031455994,
   0.6896986365318298,
   0.7747678756713867,
   0.71238774061203,
   0.8347247242927551,
   0.8716948628425598,
   0.7369875907897949,
   0.7547652125358582,
   0.781652092